<a href="https://colab.research.google.com/github/physicistgaurav/Machine_learning_ass/blob/ass3/GauravRz_57_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##COMP 484 - Machine Learning (Practical Assignment III)

###Gaurav Rizal
###Roll no: 57
###CE IV/I



In [1]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)

In [2]:
print(vectorizer)
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
  encoding='utf-8', input='content',
  lowercase=True, max_df=1.0, max_features=None, min_df=1,
  ngram_range=(1, 1), preprocessor=None, stop_words=None,
  strip_accents=None, token_pattern='(?u)\b\w\w+\b',
  tokenizer=None, vocabulary=None)

CountVectorizer()


CountVectorizer(token_pattern='(?u)\x08\\w\\w+\x08')

In [3]:
content = ["How to format my hard disk",
" Hard disk format problems "]

In [4]:
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names_out()
['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']

['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']

In [5]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


In [6]:
import sklearn.datasets
all_data = sklearn.datasets.fetch_20newsgroups(subset='all')

In [7]:

print(len(all_data.filenames))
18846
print(all_data.target_names)

18846
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
train_data = sklearn.datasets.fetch_20newsgroups(subset='train')
print(len(train_data.filenames))

11314


In [9]:
test_data = sklearn.datasets.fetch_20newsgroups(subset='test')
print(len(test_data.filenames))

7532


In [10]:
groups = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'comp.windows.x', 'sci.space']
train_data = sklearn.datasets.fetch_20newsgroups(subset='train',
  categories=groups)
print(len(train_data.filenames))

3529


In [11]:
test_data = sklearn.datasets.fetch_20newsgroups(subset='test',
  categories=groups)
print(len(test_data.filenames))

2349


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(min_df=10, max_df=0.5,
  stop_words='english', decode_error='ignore')
vectorized = vectorizer.fit_transform(train_data.data)

In [13]:

num_samples, num_features = vectorized.shape
print("#samples: %d, #features: %d" % (num_samples, num_features))

#samples: 3529, #features: 5651


#Clustering

In [14]:
num_clusters = 50

In [15]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=num_clusters, n_init=1, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration 0, inertia 5781.780937828152
Iteration 1, inertia 3197.4191227288347
Iteration 2, inertia 3168.9434563334053
Iteration 3, inertia 3155.4413719651798
Iteration 4, inertia 3145.892759623257
Iteration 5, inertia 3138.8337664607725
Iteration 6, inertia 3133.3548343372986
Iteration 7, inertia 3129.2509028470845
Iteration 8, inertia 3127.8078194401733
Iteration 9, inertia 3126.403970222413
Iteration 10, inertia 3125.305393247142
Iteration 11, inertia 3124.8296716505074
Iteration 12, inertia 3124.618886502798
Iteration 13, inertia 3124.5530964517634
Converged at iteration 13: strict convergence.


KMeans(n_clusters=50, n_init=1, random_state=3, verbose=1)

In [16]:
print("km.labels_=%s" % km.labels_)
# km.labels_=[48 23 31 ..., 6 2 22]
# print("km.labels_.shape=%s" % km.labels_.shape)
# km.labels_.shape=3529

km.labels_=[48 23 49 ... 23 10 29]


In [17]:
new_post = '''
  Disk drive problems. Hi, I have a problem with my hard disk.
  After 1 year it is working only sporadically now.
  I tried to format it, but now it doesn't boot any more.
  Any ideas? Thanks. 
'''

In [18]:
new_post_vec = vectorizer.transform([new_post])
new_post_label = km.predict(new_post_vec)[0]
new_post_label

46

In [19]:
np.array(km.labels_) == new_post_label

array([False, False, False, ..., False, False, False])

In [20]:
post_group = zip(train_data.data, train_data.target)
all = [(len(post[0]), post[0], train_data.target_names[post[1]]) for post in post_group]
graphics = sorted([post for post in all if post[2]=='comp.graphics'])
print(graphics[5])

(245, 'From: SITUNAYA@IBM3090.BHAM.AC.UK\nSubject: test....(sorry)\nOrganization: The University of Birmingham, United Kingdom\nLines: 1\nNNTP-Posting-Host: ibm3090.bham.ac.uk\n\n==============================================================================\n', 'comp.graphics')


In [21]:
noise_post = graphics[5][1]
analyzer = vectorizer.build_analyzer()
print(list(analyzer(noise_post)))

['situnaya', 'ibm3090', 'bham', 'ac', 'uk', 'subject', 'test', 'sorry', 'organization', 'university', 'birmingham', 'united', 'kingdom', 'lines', 'nntp', 'posting', 'host', 'ibm3090', 'bham', 'ac', 'uk']


In [22]:
useful = set(analyzer(noise_post)).intersection(vectorizer.get_feature_names())
print(sorted(useful))

['ac', 'birmingham', 'host', 'kingdom', 'nntp', 'posting', 'sorry', 'test', 'uk', 'united', 'university']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
for term in sorted(useful):
  print('IDF(%-10s) = %.2f' % (term,
  vectorizer._tfidf.idf_[vectorizer.vocabulary_[term]]))

IDF(ac        ) = 3.66
IDF(birmingham) = 6.77
IDF(host      ) = 1.74
IDF(kingdom   ) = 6.68
IDF(nntp      ) = 1.77
IDF(posting   ) = 1.72
IDF(sorry     ) = 4.14
IDF(test      ) = 4.33
IDF(uk        ) = 3.70
IDF(united    ) = 5.61
IDF(university) = 1.92


So, the terms with the highest discriminative power, birmingham and kingdom, are clearly not that
computer graphics related, as is the case with the terms with lower IDF scores. Understandably,
posts from different newsgroups will be clustered together.